In [1]:
# restart kernel after running this cell
!pip install pandas --upgrade

    100% |████████████████████████████████| 10.1MB 3.6MB/s 
fastai 1.0.51 has requirement numpy>=1.15, but you'll have numpy 1.14.6 which is incompatible.
  Found existing installation: pandas 0.22.0
    Uninstalling pandas-0.22.0:
      Successfully uninstalled pandas-0.22.0


In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from datetime import datetime
import os
import glob
import re
import math
import json
import random

from IPython import display
from matplotlib import cm

import tensorflow as tf
from tensorflow.python.data import Dataset

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%%time

news_train_df = pd.read_pickle('/content/drive/My Drive/BT4222 Group/workspace/news_train_df.p')

FileNotFoundError: ignored

In [3]:
%%time

market_train_df = pd.read_hdf('/content/drive/My Drive/BT4222 Group/workspace/market_train_df.h5')

CPU times: user 5.12 s, sys: 1.44 s, total: 6.56 s
Wall time: 14.3 s


In [4]:
market_train_df.columns.tolist()

['time',
 'assetCode',
 'assetName',
 'volume',
 'close',
 'open',
 'returnsClosePrevRaw1',
 'returnsOpenPrevRaw1',
 'returnsClosePrevMktres1',
 'returnsOpenPrevMktres1',
 'returnsClosePrevRaw10',
 'returnsOpenPrevRaw10',
 'returnsClosePrevMktres10',
 'returnsOpenPrevMktres10',
 'returnsOpenNextMktres10',
 'universe']

In [0]:
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from itertools import chain
import time
import multiprocessing
import copy
import pickle
import datetime as dt
import h5py
import re
from subprocess import call
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.manifold import TSNE
from sklearn.neighbors import NearestNeighbors
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.recurrent import LSTM
from keras.models import load_model
from keras import backend as K
import gc
%matplotlib inline

Using TensorFlow backend.


In [0]:
news_columns_outlier = ['takeSequence', 'bodySize', 'sentenceCount', 'wordCount', 'sentimentWordCount', 'firstMentionSentence','noveltyCount12H',\
                  'noveltyCount24H', 'noveltyCount3D', 'noveltyCount5D', 'noveltyCount7D', 'volumeCounts12H', 'volumeCounts24H',\
                  'volumeCounts3D','volumeCounts5D','volumeCounts7D']


def clip_outliers(data_frame, column_list, low=0.02, high=0.98):
    for column in column_list:
        this_column = data_frame[column]
        quant_df = this_column.quantile([low,high])
        low_limit = quant_df[low]
        high_limit = quant_df[high]
        data_frame[column] = data_frame[column].clip(lower=low_limit, upper=high_limit)
    return data_frame


def import_data(from_2009=False, sample=False, split_ratio=0.1):
    global market_train_df, news_train_df
    if sample:
        N = int(news_train_df.shape[0] * split_ratio)
        dt_end = news_train_df['time'].iloc[N]
        market_train_df = market_train_df[market_train_df['time'] <= dt_end]
        news_train_df = news_train_df[news_train_df['time'] <= dt_end]
        
    return (market_train_df, news_train_df)

In [0]:
def apply_func(f):
    '''Apply the given function on the data and close the data'''
    file = 'df_roll.h5'
    data = h5py.File(file, 'r+')
    results = f(data)
    data.close()
    return results

def get_keys():
    def f(data):
        return list(sorted(list(data['data'].keys())))
    return apply_func(f)

def get_files(w):
    res = []
    for f in os.listdir():
        if re.search('^df_%s_\d\.h5$' % w, f):
            res.append(f)
    return res

def rm_key(v):
    def helper(data):
        del data['data'][v]
    apply_func(helper)

In [0]:
# factorize columns

def reduce_news(news_train_df):
#     news_train_df.drop(['provider','subjects','audiences','sourceTimestamp','firstCreated','sourceId','headline'],axis=1,inplace=True)
#     news_train_df.drop(['provider','subjects','audiences','sourceTimestamp','firstCreated','sourceId'],axis=1,inplace=True)
    news_train_df.drop(['subjects','audiences','sourceTimestamp','firstCreated'],axis=1,inplace=True)
    
#     for column in ['sourceId', 'provider', 'headlineTag']:
#         news_train_df[column], uniques = pd.factorize(news_train_df[column])
#         del uniques
    
    # convert data types
    news_train_df['relevance']=news_train_df['relevance'].astype('float32')
    news_train_df['sentimentNegative']=news_train_df['sentimentNegative'].astype('float32')
    news_train_df['sentimentNeutral']=news_train_df['sentimentNeutral'].astype('float32')
    news_train_df['sentimentPositive']=news_train_df['sentimentPositive'].astype('float32')
    
    for c in news_train_df:
        if news_train_df[c].dtype == bool:
            news_train_df[c] = news_train_df[c].astype('int8')

In [0]:
def fill_subjects(cat_subjects_dict, news_train_df, is_test=False):
    if is_test:
        enum = enumerate(news_train_df['subjects'].cat.categories[news_train_df['subjects'].cat.codes].str.findall(f"'([\w\./]+)'"))
    else:
        enum = enumerate(news_train_df['subjects'].str.findall(f"'([\w\./]+)'"))
#     for i,ls_subjects in enumerate(news_train_df['subjects'].cat.categories[news_train_df['subjects'].cat.codes].str.findall(f"'([\w\./]+)'")):
#     for i,ls_subjects in enumerate(news_train_df['subjects'].str.findall(f"'([\w\./]+)'")):
    for i,ls_subjects in enum:
#         if i == 100000:
#             print('reached 100k')
        st_subjects = set(ls_subjects)
        for s in cat_subjects_dict:
            cat_subjects_dict[s][i] = int(s in st_subjects)

In [0]:
news_cols_agg = {
    'urgency': ['min', 'count'],
    'takeSequence': ['min', 'max'],
    'bodySize': ['mean'],
    'wordCount': ['mean'],
    'sentenceCount': ['mean'],
    'companyCount': ['mean'],
    'marketCommentary': ['mean'],
    'relevance': ['mean'],
    'sentimentNegative': ['mean'],
    'sentimentNeutral': ['mean'],
    'sentimentPositive': ['mean'],
    'sentimentWordCount': ['mean'],
    'noveltyCount12H': ['mean'],
    'noveltyCount24H': ['mean'],
    'noveltyCount3D': ['mean'],
    'noveltyCount5D': ['mean'],
    'noveltyCount7D': ['mean'],
    'volumeCounts12H': ['mean'],
    'volumeCounts24H': ['mean'],
    'volumeCounts3D': ['mean'],
    'volumeCounts5D': ['mean'],
    'volumeCounts7D': ['mean']
}

In [0]:
def convert_float_cols(df):
    for c in df:
        if df[c].dtype=='float16' or df[c].dtype=='float64': # for lgb
            df[c] = df[c].astype('float32')

            
def convert_object_to_float(data):
    for column in data:
        if data[column].dtype == 'O':
            if column in ['time', 'assetCode', 'assetName']:
                continue
            data[column] = data[column].astype('float32')


def exp_news_train_df(extra_columns, market_train_df=None, news_train_df=None):
    ls_keep = ['headline']
    
    is_test = True
    if news_train_df is None:
        is_test = False
        (market_train_df, news_train_df) = import_data()
        del market_train_df
    
    reduce_news(news_train_df)
    
    ls_extra = []
    for c in extra_columns:
        ls_extra.append(c)
        news_train_df[c] = extra_columns[c]
    
    # Split date into before and after 22h (the time used in train data)
    # E.g: 2007-03-07 23:26:39+00:00 -> 2007-03-08 00:00:00+00:00 (next day)
    #      2009-02-25 21:00:50+00:00 -> 2009-02-25 00:00:00+00:00 (current day)
    news_train_df['time'] = (news_train_df['time'] - np.timedelta64(22,'h')).dt.ceil('1D')
    
    if not is_test:
        news_train_df = news_train_df.set_index('time')
        ls_breakpoints = [None] + pd.date_range(news_train_df.index[0], news_train_df.index[-1], freq='Y').tolist() + [None]

        for f in get_files('roll'):
            call(['rm', f])
        for f in get_files('news'):
            call(['rm', f])
        for i in range(len(ls_breakpoints)-1):
            print('proc batch %d' % (i+1))
            dt1, dt2 = ls_breakpoints[i], ls_breakpoints[i+1]
            print('handling until %s' % dt2)
            dt2 = dt2-dt.timedelta(days=1) if dt2 else dt2 # handle dt2 if not None
            df_cur = news_train_df[dt1:dt2].reset_index()
            if df_cur.empty:
                continue
            df_cur['assetCodes'] = df_cur['assetCodes'].cat.categories[df_cur['assetCodes'].cat.codes].str.findall(f"'([\w\./]+)'")
            assetCodes_expanded = list(chain(*df_cur['assetCodes']))
            assetCodes_index = df_cur.index.repeat( df_cur['assetCodes'].apply(len) )
            df_assetCodes = pd.DataFrame({'level_0': assetCodes_index, 'assetCode': assetCodes_expanded})
            news_cols = ['time', 'assetCodes'] + list(news_cols_agg.keys()) + ls_extra + ls_keep
            df_cur_exp = pd.merge(df_assetCodes, df_cur[news_cols], left_on='level_0', right_index=True, suffixes=(['','_old']))
            df_cur_exp.to_hdf('df_roll_%s.h5' % i, 'data')
    else:
        # test
        news_train_df['assetCodes'] = news_train_df['assetCodes'].cat.categories[news_train_df['assetCodes'].cat.codes].str.findall(f"'([\w\./]+)'")
        assetCodes_expanded = list(chain(*news_train_df['assetCodes']))
        assetCodes_index = news_train_df.index.repeat( news_train_df['assetCodes'].apply(len) )
        df_assetCodes = pd.DataFrame({'level_0': assetCodes_index, 'assetCode': assetCodes_expanded})
        news_cols = ['time', 'assetCodes'] + sorted(news_cols_agg.keys()) + ls_extra + ls_keep
        news_train_df_expanded = pd.merge(df_assetCodes, news_train_df[news_cols], left_on='level_0', right_index=True, suffixes=(['','_old']))
        return news_train_df_expanded


def agg_news_train_df(args):
    # TODO: move logic for single agg out
    (news_train_df_expanded, extra_columns, extra_columns_agg) = args
    # Aggregate numerical news features
    news_cols_agg_local = copy.deepcopy(news_cols_agg)
    news_cols_agg_local.update(extra_columns_agg)

    if news_train_df_expanded is not None:
        # test
        df_headlines = pd.DataFrame(
            news_train_df_expanded.reset_index().groupby(['time', 'assetCode'])['headline'].agg(lambda x:'. '.join(x))
        ).reset_index()
        
        news_train_df_aggregated = news_train_df_expanded.groupby(['time', 'assetCode']).agg(news_cols_agg_local)
        news_train_df_aggregated.columns = ['_'.join(col).strip() for col in news_train_df_aggregated.columns.values]
        del news_train_df_expanded
        convert_float_cols(news_train_df_aggregated)
        return news_train_df_aggregated, df_headlines
        
    ls_h5_files = get_files('roll')
    ls_extra = list(extra_columns.keys())

    for h5f in ls_h5_files:
        h5k = re.search('\d', h5f).group(0)
        print('handling batch %s' % h5k)
        df_cur = pd.read_hdf(h5f, 'data')
        pd.DataFrame(
            df_cur.reset_index().groupby(['time', 'assetCode'])['headline'].agg(lambda x:'. '.join(x))
        ).reset_index().to_hdf('df_news_%s.h5' % h5k, 'data')
        df_cur = df_cur.reset_index().groupby(['time', 'assetCode']).agg(news_cols_agg_local)
        df_cur.columns = ['_'.join(col).strip() for col in df_cur.columns.values]
        convert_float_cols(df_cur)
        call(['rm', h5f])
        df_cur.to_hdf(h5f, 'data')


def get_market_data(market_train_df=None, news_train_df=None, skipOR=False, get_last_only=False, last_rows=None):
    if market_train_df is None and news_train_df is None:
        (market_train_df, news_train_df) = import_data()
        del news_train_df
        gc.collect()

    market_train_df['time'] = market_train_df['time'].dt.floor('1D')
    
    market_train_df['open_close'] = market_train_df['close'] / market_train_df['open'] - 1
    market_train_df['oc_average'] = (market_train_df['close'] + market_train_df['open']) / 2
    market_train_df['turnover'] = market_train_df['close'] * market_train_df['volume']

    relative_feature_names = [
        'open_close',
        'turnover',
        'volume',
        'returnsOpenPrevMktres1',
        'returnsOpenPrevMktres10',
    ]
    
    print('generating relative features..')
    if get_last_only:
        if market_train_df.shape[0] >= 2:
            dt_last = market_train_df['time'].iloc[-1]
            market_train_df = market_train_df.set_index('time').loc[dt_last, :]

            df_relative = market_train_df[relative_feature_names].rename(columns = {
                feature_name: feature_name + '_relative' for feature_name in relative_feature_names
            })
            df_relative = (df_relative - df_relative.mean(axis=0)) / df_relative.std(axis=0)
            market_train_df = pd.concat([market_train_df, df_relative], axis=1)
            
    else:
        market_grouped = market_train_df.set_index('time').groupby('time')
        original_index = market_train_df.set_index('time').index
        del market_train_df
        market_train_df = None

        count = 0
        for idx, df_time in market_grouped:
            count += 1
            if count % 1000 == 0:
                print('%dth day..' % count)
            if df_time.shape[0] < 2:
                continue
            df_relative = df_time[relative_feature_names].rename(columns = {
                feature_name: feature_name + '_relative' for feature_name in relative_feature_names
            })
            df_relative = (df_relative - df_relative.mean(axis=0)) / df_relative.std(axis=0)
            df_with_relative = pd.concat([df_time, df_relative], axis=1)

            if market_train_df is None:
                market_train_df = pd.DataFrame(
                    index = original_index,
                    columns = df_with_relative.columns
                )
            market_train_df.loc[idx, :] = df_with_relative.values
        del market_grouped
        gc.collect()
    
    # reset index
    market_train_df = market_train_df.reset_index()
    
    print('[relative] Converting object columns to float..')
    convert_object_to_float(market_train_df)
    print('Finished converting..')
    
    print('finished creating features..')
    
    if 'returnsOpenNextMktres10' in market_train_df.columns:
        market_train_df['returnsOpenNextMktres10'] = market_train_df['returnsOpenNextMktres10'].astype('float32')
    if 'universe' in market_train_df.columns:
        market_train_df['universe'] = market_train_df['universe'].astype('int8')

    return market_train_df

In [0]:
%%time

def save_market_data(_):
    market_data = get_market_data(skipOR=False)
    market_data = market_data[~market_data['assetCode'].isin(['PGN.N','EBRYY.OB'])]
    if 'df_market_data.h5' in os.listdir():
        !rm df_market_data.h5
    market_data.to_hdf('df_market_data.h5', 'data')

with multiprocessing.Pool(1) as pool:
    pool.map(save_market_data, [None])

generating relative features..
1000th day..
2000th day..
[relative] Converting object columns to float..
Finished converting..
finished creating features..


Process ForkPoolWorker-1:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "<timed exec>", line 6, in save_market_data
  File "/usr/local/lib/python3.6/dist-packages/google/colab/_shell.py", line 67, in system
    output = _system_commands._system_compat(self, *args, **kwargs)  # pylint:disable=protected-access


KeyboardInterrupt: ignored

In [0]:
%%time

with multiprocessing.Pool(1) as pool:
    pool.map(exp_news_train_df, [{}])[0]

proc batch 1
handling until 2007-12-31 00:00:00+00:00


/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:2378: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block6_values] [items->['assetCode', 'assetCodes', 'headline']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


proc batch 2
handling until 2008-12-31 00:00:00+00:00
proc batch 3
handling until 2009-12-31 00:00:00+00:00
proc batch 4
handling until 2010-12-31 00:00:00+00:00
proc batch 5
handling until 2011-12-31 00:00:00+00:00
proc batch 6
handling until 2012-12-31 00:00:00+00:00
proc batch 7
handling until 2013-12-31 00:00:00+00:00
proc batch 8
handling until 2014-12-31 00:00:00+00:00
proc batch 9
handling until 2015-12-31 00:00:00+00:00
proc batch 10
handling until None
CPU times: user 93.6 ms, sys: 105 ms, total: 198 ms
Wall time: 1min 20s


In [0]:
%%time

with multiprocessing.Pool(1) as pool:
    pool.map(agg_news_train_df, [(None, {}, {})])[0]

handling batch 3
handling batch 6
handling batch 1
handling batch 7
handling batch 4
handling batch 9
handling batch 5
handling batch 0
handling batch 2
handling batch 8
CPU times: user 191 ms, sys: 151 ms, total: 342 ms
Wall time: 3min 45s


In [0]:
def label_encode(series, min_count, encoder_dict):
    vc = series.value_counts()
    max_value = max(list(encoder_dict.values)) if len(encoder_dict) else 0
    for i, c in enumerate(vc.index[vc >= min_count]):
        if c in encoder_dict:
            continue
        encoder_dict[c] = max_value + i
    return encoder_dict


def make_test(market_train_df, news_train_df_aggregated, df_headlines):
    df_xtr = market_train_df.join(news_train_df_aggregated, on=['time', 'assetCode']) 
    df_xtr = df_xtr.join(df_headlines.set_index(['time', 'assetCode']), on=['time', 'assetCode'])
    
    df_xtr['dayofweek'], df_xtr['month'] = df_xtr.time.dt.dayofweek, df_xtr.time.dt.month
#     ls_todrop = ['assetName']
    ls_todrop = []
    df_xtr.drop(ls_todrop, axis=1, inplace=True)
    return df_xtr


def build_train(_):
    # for train
    if 'df_roll.h5' in os.listdir():
        !rm df_roll.h5
    if 'df_market_data.h5' not in os.listdir():
        raise Exception('Create market data first')
    market_train_df = pd.read_hdf('df_market_data.h5', 'data')
    market_train_df['time'] = market_train_df['time'].dt.tz_localize(None)
    market_train_df = market_train_df.set_index('time')
    ls_xtr = []
    encoder_dict = {}
    for h5f in sorted(get_files('roll')):
        h5k = re.search('\d', h5f).group(0)
        print('proc batch %s' % h5k)
        df_agg = pd.read_hdf(h5f, 'data')
        df_news = pd.read_hdf('df_news_%s.h5' % h5k, 'data')
        df_news['time'] = df_news['time'].dt.tz_localize(None)
        dt1, dt2 = df_agg.index[0][0], df_agg.index[-1][0]
        df_xtr = market_train_df[dt1:dt2].reset_index().join(df_agg, on=['time', 'assetCode'])
        df_xtr = df_xtr.join(df_news.set_index(['time', 'assetCode']), on=['time', 'assetCode'])
        
        # keep assetName
#         df_xtr.drop('assetName', axis=1, inplace=True)
        
        df_xtr['dayofweek'], df_xtr['month'] = df_xtr.time.dt.dayofweek, df_xtr.time.dt.month
        ls_xtr.append(df_xtr)
        call(['rm', h5f])
        call(['rm', 'df_news_%s.h5' % h5k])
    df_xtr = pd.concat(ls_xtr)
    
    # create headline features
    # create headline features later to avoid leakage
#     headline_features = transform_headlines(df_xtr['headline'])
#     df_xtr = df_xtr.drop(columns=['headline'])
#     for column in headline_features:
#         df_xtr[column] = headline_features[column]

    le_assetCode = label_encode(df_xtr['assetCode'], 10, encoder_dict)
#     df_xtr.to_hdf('df_roll.h5', 'data')
#     df_xtr.to_pickle('df_roll.p')
    return le_assetCode, df_xtr

#     return le_assetCode, le_assetName

In [0]:
%%time

with multiprocessing.Pool(1) as pool:
#     le_assetCode, le_assetName = pool.map(build_train, [None])[0]
    le_assetCode, df_xtr = pool.map(build_train, [None])[0]

proc batch 0
proc batch 1
proc batch 2
proc batch 3
proc batch 4
proc batch 5
proc batch 6
proc batch 7
proc batch 8
proc batch 9
CPU times: user 2.17 s, sys: 3.21 s, total: 5.38 s
Wall time: 1min 7s


In [0]:
%%time
df_xtr.to_pickle('/content/drive/My Drive/BT4222 Group/workspace/data/combined.p')

### Remark
* The following code blocks are related to model building instead of preprocessing

In [0]:
def sigma_score(preds, valid_data):
    df_time = valid_data.params['extra_time']
    labels = valid_data.get_label()
    
    assert len(labels) == len(df_time)
    
    x_t = preds * labels

    x_t_sum = x_t.groupby(df_time).sum()
    score = x_t_sum.mean() / x_t_sum.std()
    return 'sigma_score', score, True


def sigma_score_plain(preds, labels, ds_time):
    x_t = preds * labels
    x_t_sum = x_t.groupby(ds_time.values).sum()
    score = x_t_sum.mean() / x_t_sum.std()
    return score

In [0]:
lgb_params = dict(
    objective = 'regression_l1',
    learning_rate = 0.1,
    num_leaves = 12,
    max_depth = 50,
#     min_data_in_leaf = 1000,
#     min_sum_hessian_in_leaf = 10,
    bagging_fraction = 0.75,
    bagging_freq = 2,
    feature_fraction = 0.5,
    lambda_l1 = 0.0,
    lambda_l2 = .1,
    metric = 'l2', # bbb
    seed = 42
)

# potential area for improvement


class BaseModel():
    def __init__(self):
        self.model = None
        self.xtr = None
        self.ytr = None
        self.xval = None
        self.yval = None
        self.lgb_params = lgb_params
        self.learning_rate_decay_fn = learning_rate_power
        
    
    
    def fit(self, xtr, ytr):
        raise Exception('Not implemented. Override this method.')
    
    
    def predict(self, xval):
        raise Exception('Not implemented. Override this method.')
        


class LGBBasicModel(BaseModel):
    def __init__(self, ban_words=None):
        self.categorical_cols = ['dayofweek', 'month']
        self.lgb_params = lgb_params
        self.remove_list = ['time', 'universe', 'returnsOpenNextMktres10', 'assetCode', 'assetName', 'headline']
        self.ban_words = ban_words
        

    def fit_eval(self, xtr, ytr, xval, yval, num_rounds=None):
        self.fit_full(xtr, ytr, num_rounds)
        return self.predict(xval)
        
        
    def _filter_regex(self, data):
        return data
    
    
    def predict(self, xval):
        ls_to_drop = []
            
        for c in self.remove_list:
            if c in xval.columns:
                ls_to_drop.append(c)
        xval = xval.drop(columns=ls_to_drop)
            
        xval = self._filter_regex(xval)
        
        xval = xval[self.train_cols]
        return self.model.predict(xval)
    
    
    def fit(self, xtr, ytr):
        if self.ban_words is not None:
            drop_list = []
            for col in xtr.columns:
                for ban_word in self.ban_words:
                    if ban_word in col:
                        drop_list.append(col)
                        break
            xtr = xtr.drop(columns=drop_list)
            
        # actual training and validation sets
        xtr_, xval_, ytr_, yval_ = train_test_split(xtr, ytr, test_size=0.2, shuffle=False)
        
        valid_params = {
            'extra_time': xval_['time'].factorize()[0]
        }
        
        ls_to_drop = []
        for col in self.remove_list:
            if col in xtr_.columns:
                ls_to_drop.append(col)
        
        xtr_ = xtr_.drop(columns=ls_to_drop)
        xval_ = xval_.drop(columns=ls_to_drop)
        
        xtr_ = self._filter_regex(xtr_)
        xval_ = self._filter_regex(xval_)
        
        train_cols = xtr_.columns.tolist()
        
        # Note: y data is expected to be a pandas Series, as we will use its group_by function in `sigma_score`
        dtrain = lgb.Dataset(xtr_.values, ytr_, feature_name=train_cols, categorical_feature=self.categorical_cols, free_raw_data=False)
        dvalid = lgb.Dataset(xval_.values, yval_, feature_name=train_cols, categorical_feature=self.categorical_cols, free_raw_data=False)
        dvalid.params = valid_params
        evals_result = {}
        m = lgb.train(lgb_params, dtrain, num_boost_round=1000, valid_sets=(dvalid,), valid_names=('valid',), verbose_eval=25,
                      early_stopping_rounds=100, evals_result=evals_result)
        self.train_cols = train_cols
        self.model = m
        self.evals_result = evals_result
    
    
    def fit_full(self, xtr, ytr, num_rounds=None):
        if self.ban_words is not None:
            drop_list = []
            for col in xtr.columns:
                for ban_word in self.ban_words:
                    if ban_word in col:
                        drop_list.append(col)
                        break
            xtr = xtr.drop(columns=drop_list)
        
        if num_rounds is None:
            df_result = pd.DataFrame(self.evals_result['valid'])
            num_rounds = df_result['l2'].argmax()+1
        
        ls_to_drop = []
        for col in self.remove_list:
            if col in xtr.columns:
                ls_to_drop.append(col)
        
        xtr = xtr.drop(columns=ls_to_drop)
        xtr = self._filter_regex(xtr)
        
        train_cols = xtr.columns.tolist()
        dtrain = lgb.Dataset(xtr.values, ytr, feature_name=train_cols, categorical_feature=self.categorical_cols, free_raw_data=False)
        
        m = lgb.train(self.lgb_params, dtrain, num_boost_round=num_rounds)
        self.train_cols = train_cols
        self.model = m

In [0]:
%%time

def get_stack_data(models):
#     df_xtr = pd.read_hdf('df_roll.h5', 'data')
#     df_xtr = pd.read_pickle('df_roll.p')
#     df_xtr = df_xtr.drop(columns='headline')
    
    global df_xtr
    
    arr_dates = df_xtr['time'].unique()
    dt_start = arr_dates[-(arr_dates.shape[0] // 504) * 504]
    
    df_xtr = df_xtr[df_xtr['time'] >= dt_start]
    
    target = df_xtr['returnsOpenNextMktres10'].copy()
#     stack_data = stack_models(df_xtr, models)
    
    stack_data = df_xtr
    
    stack_data['sentimentPositive_outperform'] = stack_data['sentimentPositive_mean'] - stack_data['sentimentPositive_mean'].mean()
    stack_data['sentimentNegative_outperform'] = stack_data['sentimentNegative_mean'] - stack_data['sentimentNegative_mean'].mean()
#     stack_data['open_close_outperform'] = stack_data['open_close'] - stack_data['open_close'].mean()
#     stack_data['returnsOpenPrevMktres1_outperform'] = stack_data['returnsOpenPrevMktres1'] - stack_data['returnsOpenPrevMktres1'].mean()
#     stack_data['returnsOpenPrevMktres10_outperform'] = stack_data['returnsOpenPrevMktres10'] - stack_data['returnsOpenPrevMktres10'].mean()

#     stack_data.to_hdf('stack_data.h5', 'data')
#     stack_data.to_pickle('stack_data.p')
#     return stack_data, target
    return stack_data

with multiprocessing.Pool(1) as pool:
    df_stack = pool.map(get_stack_data, [None])[0]

In [0]:
def cv_stack(X, model, k=5, seq=True, cv_type='model'):
    if seq:
        k += 1
        breakpoints = [int(float(X.shape[0])*i/float(k)) for i in range(1, k)] + [X.shape[0]]
    else:
        breakpoints = [int(float(X.shape[0])*i/float(k)) for i in range(k)] + [X.shape[0]]
    X = X.set_index('time')
    y = X['returnsOpenNextMktres10']
    X = X.drop(columns=['returnsOpenNextMktres10'])
    res = []
    
    for i in range(len(breakpoints)-1):
        # skip to last round (remove)
        if i+1 < len(breakpoints)-1:
            continue
            
        p1, p2 = breakpoints[i], breakpoints[i+1]
        dt1, dt2 = X.index[p1], X.index[p2] if p2 < X.shape[0] else None
        
        dt1, dt2 = pd.to_datetime(dt1), pd.to_datetime(dt2)
        X_train = X[:dt1 - dt.timedelta(days=1)].reset_index()
        y_train = y[:dt1 - dt.timedelta(days=1)]
        X_test = X[dt1:dt2].reset_index()
        y_test = y[dt1:dt2]
        ds_time = X[dt1:dt2].index
        
        
        print('Fold %d (%s to %s)' % (i+1, dt1, dt2))
        if seq:
            if cv_type == 'model':
                model.fit_full(X_train, y_train, num_rounds=200)
                preds = model.predict(X_test)
            elif cv_type == 'average':
                preds = X_test.mean(axis=1)
            elif cv_type == 'majority':
                preds = []
                for row in X_test.values:
                    if (row > 0).sum() > X.shape[1] / 2:
                        preds.append(row[row>0].mean())
                    else:
                        preds.append(row[row<=0].mean())
            else:
                raise Exception('Unsupported')
            score = sigma_score_plain(np.array(preds), y_test, ds_time)
            res.append(score)
        else:
            raise Exception('Not supported')
            arr_flags = (np.arange(X.shape[0]) < dt1) | (np.arange(X.shape[0]) >= dt2)
            res.append(fit_val
                       (X[arr_flags], y[arr_flags], X.iloc[dt1:dt2], y.iloc[dt1:dt2]))
    return res

In [0]:
%%time

cv_results = cv_stack(df_stack, LGBBasicModel(), cv_type='model')

NameError: ignored

In [0]:
cv_results

NameError: ignored